# EDA for List_of_mountain_lakes_of_Switzerland

In [1]:
import pandas as pd
import requests # library to handle requests
from bs4 import BeautifulSoup
import re
import shutil
from fake_useragent import UserAgent

In [2]:
df = pd.read_csv("mountain_lake.csv")
df.head()

,lake_id,Lake,Canton(s),Elevation in m[2],Area in ha[3],Basin highest point[4],Basin highest elevation in m,River basin,is_lake_link,link,latlng,with_geojson
0,1,Theodulgletschersee,Valais,2851,7.76,Theodulhorn,3469,Rhone,False,https://en.wikipedia.org///wiki/Theodulgletsch...,"[45.944444, 7.716667]",True
1,2,Lai d'Immez,Graubünden,2832,4.53,Piz d'Immez,3033,Danube,False,NaN,"[46.7277984, 10.1338334]",True
2,3,Lej da Prüna,Graubünden,2815,4.03,West of Piz Pischa,3080,Danube,False,NaN,"[46.818188, 8.227511999999999]",True
3,4,Lej da Pischa,Graubünden,2770,5.05,Piz Pischa,3138,Danube,False,NaN,"[46.47142909999999, 9.9790387]",True
4,5,Chaltwassersee,Valais,2756,4.41,Monte Leone,3553,Rhone,False,NaN,"[46.25839999999999, 8.078289999999999]",False


The links are not all valid, it can be page not exist or a page of mountain/glacier. Links to page that talks about mountain/glacier but not related to lake/dam/reservoir at all are abandoned. For page of mountain/glacier but keywords like lake/dam/reservoir are contained in the passage, is_lake_link is False but the link will be stored. Only for papge of the lake/dam/reservoir the is_lake_link is True and the link is stored.

In [3]:
df.count()

Unnamed: 0                      217
Lake                            217
Canton(s)                       217
Elevation in m[2]               217
Area in ha[3]                   217
Basin highest point[4]          217
Basin highest elevation in m    217
River basin                     217
is_lake_link                    217
link                            168
lake_id                         217
dtype: int64

In [4]:
df['is_lake_link'].value_counts()

True     154
False     63
Name: is_lake_link, dtype: int64

So we have 168 links in total for 217 lakes. However only 154 links are to the lake's own page, the rest 14 links to other content but mention the lake, in which NLP can be applied later to extract information of the lake.

In [5]:
df.groupby('Canton(s)').size()

Canton(s)
Appenzell Innerrhoden     3
Bern                     22
Fribourg                  2
Glarus                    7
Graubünden               63
Graubünden/Ticino         1
Jura                      1
Neuchâtel                 1
Nidwalden                 2
Obwalden                  5
Schwyz                    4
St. Gallen                8
Ticino                   33
Ticino/Valais             1
Uri                       9
Valais                   48
Vaud                      7
dtype: int64

### Alpine Lakes (located above 2000 meters)

In [6]:
# change the elevation from string to int
df['Elevation in m[2]'] = df['Elevation in m[2]'].astype(int)

In [7]:
df_2000 = df[df['Elevation in m[2]']>2000]
df_2000.count()

Unnamed: 0                      101
Lake                            101
Canton(s)                       101
Elevation in m[2]               101
Area in ha[3]                   101
Basin highest point[4]          101
Basin highest elevation in m    101
River basin                     101
is_lake_link                    101
link                             65
lake_id                         101
dtype: int64

In [8]:
df_2000['is_lake_link'].value_counts()

True     53
False    48
Name: is_lake_link, dtype: int64

## Collect info from lake page

First look at the lake's page, i.e. is_lake_link = True, then go to the page. Collect info like max length, max width, max depth, water volume, coordination, img, type.

In [67]:
lake_id = [] # refers to the previous dataframe
lake_img = [] # store a list of links to the imgs for each lake also redirect to the wikimedia page if it exists!!
coordinates = []
lake_type = []
max_length = []
max_width = []
max_depth = []
#average_depth = []
water_volume = []
if_external_link = []

text = [] # store the text in the wiki entry of the lake

for index, row in df.iterrows():
    if(row['is_lake_link'] == True):
        lake_id.append(row['lake_id'])
        
        response = requests.get(row['link'])
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # extract the plain text content from paragraphs
        this_text = ''
        for paragraph in soup.find('div',{'class':'mw-parser-output'}).find_all('p',{'class':''}):
            this_text += paragraph.text
        # clean text 
        this_text = re.sub(r'\[.*?\]+', '', this_text)
        this_text = this_text.replace('\n', '')
        text.append(this_text)
        
        # get images from wikimeida
        this_imgs = []
        for external_link in soup.find_all('a',href=True):
            if('https://commons.wikimedia.org/' in str(external_link)):
                # go to the wiki media page and scrap all images, store the link in the list
                wikimedia_response = requests.get(external_link['href'])
                wikimedia_soup = BeautifulSoup(wikimedia_response.text, 'html.parser')
                for i in wikimedia_soup.find_all('img',src=True):
                    img_height = i.get('height')
                    img_link = i.get('src')
                    if('upload.wikimedia.org' not in img_link):
                        continue
                    # discard map
                    if('_map.' in img_link):
                        continue
                    if(img_height and int(img_height)>50):
                        # get link to the original pictures
                        new_link = img_link.replace("thumb/", "")
                        new_link = new_link[:new_link.rindex('/')]
                        if(new_link not in this_imgs):
                            this_imgs.append(new_link)
                
        
        table = soup.find_all('table',{'class':"infobox vcard"})[0]
        #print(table)
        #get coordinates
        latitude = None
        longitude = None
        if(table.find('span', {'class':'latitude'})):
            latitude = table.find('span', {'class':'latitude'}).text
            longitude = table.find('span', {'class':'longitude'}).text
        coordinates.append((latitude, longitude))
        
        #get images from current page
        for raw_img in soup.find_all('img'):
            # if the height of the img is smaller than 100 it is probably a icon 
            img_height = raw_img.get('height')
            img_link = raw_img.get('src')
            if('upload.wikimedia.org' not in img_link):
                continue
            # discard map
            if('_map.' in img_link):
                continue
            if(img_height and int(img_height)>50):
                # get link to the original pictures
                new_link = img_link.replace("thumb/", "")
                new_link = new_link[:new_link.rindex('/')]
                if(new_link not in this_imgs):
                    this_imgs.append(new_link)
                
        
        this_type = 'NA'
        this_max_length = 'NA'
        this_max_width = 'NA'
        this_max_depth = 'NA'
        this_water_volume = 'NA'
        
        # first tr is the title skip it
        for row in table.find_all('tr')[1:]:
            if(not row.find('th')):
                continue
            elif('Type' in row.find('th').text):
                this_type = row.find('td').text
            elif('Max. length' in row.find('th').text):
                this_max_length = row.find('td').text
            elif('Max. width' in row.find('th').text):
                this_max_width = row.find('td').text
            elif('Max. depth' in row.find('th').text):
                this_max_depth = row.find('td').text
            elif('Water volume' in row.find('th').text):
                this_water_volume = row.find('td').text
        
        this_external_link = False
        if(soup.find('span', {'id': 'External_links'})):
            this_external_link = True
        
        
        lake_type.append(this_type)
        lake_img.append(this_imgs)
        max_length.append(this_max_length)
        max_width.append(this_max_width)
        max_depth.append(this_max_depth)
        water_volume.append(this_water_volume)
        if_external_link.append(this_external_link)
        

In [ ]:
da = {'lake_id': lake_id, 'description': text, 'lake_img': lake_img, 'coordinates': coordinates, 'lake_type':lake_type,
       'max_length':max_length, 'max_width':max_width, 'max_depth':max_depth, 'water_volume':water_volume,
       'if_external_link':if_external_link}
df2 = pd.DataFrame(data=da)

In [69]:
df2.to_csv('placeholder.csv',index=False)

In [70]:
df2['if_external_link'].value_counts()

True     101
False     53
Name: if_external_link, dtype: int64

## Download Images from the links

In [ ]:
df2 = pd.read_csv("from_lake_entry.csv")

In [14]:
FAKE_HEADERS = {'User-Agent': UserAgent().random}
        
for index, row in df2.iterrows():
    links = row['new_imgs'][1:-1]
    if(len(links)<2 or row['lake_id']<143):
        continue
    links = links.split(', ')
    for link in links:
        #print(links[1:-1])
        link = link[1:-1]
        if(not link.startswith('http')):
            link = 'https:'+link
            
        res = requests.get(link, headers=FAKE_HEADERS, stream = True)      
        name = str(row['lake_id']) + '_' + link.split('/')[-1]
        #print(res.status_code)
        if(res.status_code == 200):
            res.raw.decode_content = True
            
            with open(name,'wb') as f:
                    shutil.copyfileobj(res.raw, f)
        else:
            print(link, name)

## NLP

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

#### NER

In [16]:
sent = nltk.word_tokenize(df2['description'][10])
sent = nltk.pos_tag(sent)

ne = nltk.ne_chunk(sent,binary=True)
print(ne)

(S
  (NE Bortelsee/NNP)
  is/VBZ
  a/DT
  reservoir/NN
  in/IN
  the/DT
  municipality/NN
  of/IN
  (NE Brig/NNP)
  in/IN
  the/DT
  canton/NN
  of/IN
  (NE Valais/NNP)
  ,/,
  (NE Switzerland/NNP)
  ./.
  Its/PRP$
  dam/NN
  was/VBD
  built/VBN
  in/IN
  1989/CD
  ./.)


In [11]:
df2['description'][10]

'Bortelsee is a reservoir in the municipality of Brig in the canton of Valais, Switzerland. Its dam was built in 1989.'

In [10]:
doc = nlp(df2['description'][10])
print([(X.text, X.label_) for X in doc.ents])

[('Bortelsee', 'GPE'), ('Brig', 'PERSON'), ('Valais', 'GPE'), ('Switzerland', 'GPE'), ('1989', 'DATE')]


#### Summary

In [20]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from heapq import nlargest

In [21]:
stopwords = stopwords.words('english')
punc = punctuation+'\n'

In [ ]:
text = df2['description'][13] #########
tokens = word_tokenize(text)
word_frequencies = {}
for i in tokens:
    t = i.lower()
    if t not in punc and t not in stopwords:
        if t not in word_frequencies.keys():
            word_frequencies[t] = 1
        else:
            word_frequencies[t] += 1

In [35]:
max_freq = max(word_frequencies.values())
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_freq


In [36]:
sent_tokens = sent_tokenize(text)
sent_weights = {}
for sent in sent_tokens:
    for word in word_tokenize(sent.lower()):
        if(word in word_frequencies.keys()):
            if(sent not in sent_weights.keys()):
                sent_weights[sent] = word_frequencies[word]
            else:
                sent_weights[sent] += word_frequencies[word]
        
sent_weights        

{'Geisspfadsee is a lake in Valais, Switzerland.': 2.5,
 'Its surface area is 18\xa0ha (44 acres).': 3.0,
 'The lake is located in the municipality of Binn, at an elevation of 2439 m. It drains into Züesee (2420 m).': 5.5,
 'The nearby border to Italy can be crossed at two mountain passes: Geisspfadpass (Passo della Rossa) and Grampielpass (Passo di Crampiolo).': 8.5,
 "The name 'geisspfad' means, only goat may pass.": 4.0,
 "A 'geiss' is a female goat in local mother tongue.": 3.5,
 'Smugglers used this pass in recent times too, but it was never accessible with horses or donkeys for payloads.': 5.5}

In [37]:
select_length = int(len(sent_weights)*0.3)
summary = nlargest(select_length, sent_weights, key = sent_weights.get)
final_summary = [word for word in summary]
summa = ''.join(final_summary)
summa

'The nearby border to Italy can be crossed at two mountain passes: Geisspfadpass (Passo della Rossa) and Grampielpass (Passo di Crampiolo).The lake is located in the municipality of Binn, at an elevation of 2439 m. It drains into Züesee (2420 m).'

# Get LatLng for all lakes from google API

In [49]:
latlng = []
for index, row in df.iterrows():
    name = row['Lake']
    this_ll = None
    qry = name.replace(' ', '+')
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+qry+',+Switzerland&key=')
    resp_json_payload = response.json()
    if(resp_json_payload['status']=='OK'):
        for i in resp_json_payload['results']:
            if('geometry' in i):
                if(i['geometry']['location']):
                    this_ll = [i['geometry']['location']['lat'],i['geometry']['location']['lng']]
                    break
        
    latlng.append(this_ll)

## Lakes in the SiwssTopo dataset

In [130]:
import geojson
with open('lake.geojson') as f:
    gj = geojson.load(f)

In [105]:
gj['features'][0]

{"geometry": {"coordinates": [[[7.93612, 47.397061], [7.936144, 47.397073], [7.936159, 47.397069], [7.936166, 47.39706], [7.936184, 47.397064], [7.936217, 47.397046], [7.936292, 47.39701], [7.93634, 47.396981], [7.93637, 47.396948], [7.936372, 47.396931], [7.936357, 47.396922], [7.9363, 47.396915], [7.936241, 47.39691], [7.936197, 47.396911], [7.936165, 47.396914], [7.936155, 47.396932], [7.93613, 47.396978], [7.936121, 47.397039], [7.93612, 47.397061]]], "type": "Polygon"}, "properties": {"name": "Falkensteinweiher"}, "type": "Feature"}

In [106]:
len(gj['features'])

1263

In [107]:
# all lakes that have geotopo info
lake_names = []
for lake in gj['features']:
    lake_names.append(lake['properties']['name'])

In [115]:
data = pd.read_csv("full_data.csv")

In [109]:
with_geojson = []
for index, row in data.iterrows():
    name = row['Lake']
    if(name in lake_names):
        with_geojson.append(True)
    else:
        with_geojson.append(False)

In [110]:
sum(with_geojson)

164

In [111]:
data['with_geojson'] = with_geojson

In [112]:
df_nogeo = data[data['with_geojson']==False]

In [114]:
data.to_csv('full_data.csv',index=False)

## Conbine CSVs into one that can be used on visualization

In [53]:
df = pd.read_csv("WikiLakesCorrectCoord.csv")
df = df.drop(['Basin highest point', 'Basin highest elevation in m', 'River basin'], axis=1)
df.head()

,lake_id,Lake,Canton,Elevation in m,Area in ha,is_lake_link,link,latlng,with_geojson
0,1,Theodulgletschersee,Valais,2851,7.76,False,https://en.wikipedia.org///wiki/Theodulgletsch...,"[45.971600, 7.715315]",True
1,2,Lai d'Immez,Graubünden,2832,4.53,False,NaN,"[46.7277984, 10.1338334]",True
2,3,Lej da Prüna,Graubünden,2815,4.03,False,NaN,"[46.477777, 9.973627]",True
3,4,Lej da Pischa,Graubünden,2770,5.05,False,NaN,"[46.47142909999999, 9.9790387]",True
4,5,Chaltwassersee,Valais,2756,4.41,False,NaN,"[46.25839999999999, 8.078289999999999]",False


In [54]:
df2 = pd.read_csv("from_lake_entry.csv")
df2 = df2.drop(['lake_img','coordinates','if_external_link'], axis=1)
df2.head()

,lake_id,description,lake_type,max_length,max_width,max_depth,water_volume
0,10,Lac du Grand Désert is a lake in the canton of...,Glacial lake,NaN,NaN,NaN,NaN
1,11,Lac de Lona is a lake at an elevation of 2640 ...,NaN,NaN,NaN,NaN,NaN
2,14,"Lej da la Tscheppa is a lake above Sils Maria,...",NaN,NaN,NaN,32 m (105 ft),NaN
3,17,Distelsee is a high alpine lake in the canton ...,natural freshwater lake,"320 metres (1,050 ft)",265 metres (869 ft),NaN,NaN
4,19,Trützisee is a lake above Geschinen in the Can...,NaN,NaN,NaN,NaN,NaN


In [55]:
con = df.join(df2.set_index('lake_id'), on='lake_id')
con.head()

,lake_id,Lake,Canton,Elevation in m,Area in ha,is_lake_link,link,latlng,with_geojson,description,lake_type,max_length,max_width,max_depth,water_volume
0,1,Theodulgletschersee,Valais,2851,7.76,False,https://en.wikipedia.org///wiki/Theodulgletsch...,"[45.971600, 7.715315]",True,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Lai d'Immez,Graubünden,2832,4.53,False,NaN,"[46.7277984, 10.1338334]",True,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Lej da Prüna,Graubünden,2815,4.03,False,NaN,"[46.477777, 9.973627]",True,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Lej da Pischa,Graubünden,2770,5.05,False,NaN,"[46.47142909999999, 9.9790387]",True,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Chaltwassersee,Valais,2756,4.41,False,NaN,"[46.25839999999999, 8.078289999999999]",False,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df3 = pd.read_csv("sampled_lakes.csv")
df3 = df3.drop(['Sample acronym', 'Filtered for extraction (L)', 'Comments', 'Filtration', 'Extraction', 
                'Inactivation', 'Ratio 18/16', 'g23 (gc/l)',
                'Sample taken for Jasmine', 'New extraction', 'Volume filtré new extraction ml', 
                'Acid nucleic concentration ng/ul', 'NA contained', 'How many could be used for 18S again'], axis=1)
#df3.lake_id = df3.lake_id.astype(int)
df3.head()

,lake_id,Lake,Region,Date,Altitude (m),pH,Conductivity (uS/cm),Temperature (°C),O2 (mg/l)
0,218,Stellisee,Zermatt,02.09.21,2538,"8,65",316,"13,3","9,53"
1,219,Grindjesee,Zermatt,03.09.21,2334,"8,18",265,"12,8","8,65"
2,220,Grünsee,Zermatt,04.09.21,2303,"8,93",315,"17,1","8,53"
3,221,Mossjesee,Zermatt,05.09.21,2139,"8,08",295,13,"8,81"
4,222,Leisee,Zermatt,06.09.21,2232,"9,25",250,"18,5","8,88"


In [57]:
con = con.merge(df3, how='outer', left_on=['lake_id','Lake'], right_on=['lake_id','Lake'])

In [58]:
con

,lake_id,Lake,Canton,Elevation in m,Area in ha,is_lake_link,link,latlng,with_geojson,description,...,max_width,max_depth,water_volume,Region,Date,Altitude (m),pH,Conductivity (uS/cm),Temperature (°C),O2 (mg/l)
0,1,Theodulgletschersee,Valais,2851.0,7.76,False,https://en.wikipedia.org///wiki/Theodulgletsch...,"[45.971600, 7.715315]",True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Lai d'Immez,Graubünden,2832.0,4.53,False,NaN,"[46.7277984, 10.1338334]",True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Lej da Prüna,Graubünden,2815.0,4.03,False,NaN,"[46.477777, 9.973627]",True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Lej da Pischa,Graubünden,2770.0,5.05,False,NaN,"[46.47142909999999, 9.9790387]",True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Chaltwassersee,Valais,2756.0,4.41,False,NaN,"[46.25839999999999, 8.078289999999999]",False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,234,Petit Aletsch 1 (PA1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Aletsch,16.10.21,2350.0,"7,67","64,3","6,8","9,99"
233,235,Petit Aletsch 2 (PA2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Aletsch,17.10.21,2350.0,"7,67","12,8","6,3","10,43"
234,236,Petit Aletsch 3 (PA3),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Aletsch,18.10.21,2335.0,"8,21","60,4","4,9","11,07"
235,237,Marjelensee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Aletsch,19.10.21,2302.0,"8,14","57,3","4,4","10,35"


In [84]:
con.to_csv('data.csv',index=False)

In [85]:
# fill missing cells for extra sampled lakes
for i, row in con.iterrows():
    if(int(row['lake_id'])>217):
        print(row['lake_id'])
        con.at[i,'Elevation in m'] = row['Altitude (m)']
        con.at[i,'Canton'] = 'Valais'
        con.at[i,'is_lake_link'] = False
        
        name = row['Lake']
        this_ll = None
        qry = name.replace(' ', '+')
        response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+qry+',+Switzerland&key=')
        resp_json_payload = response.json()
        if(resp_json_payload['status']=='OK'):
            for i in resp_json_payload['results']:
                if('geometry' in i):
                    if(i['geometry']['location']):
                        this_ll = [i['geometry']['location']['lat'],i['geometry']['location']['lng']]
                        break
        print(this_ll)
        #con.loc[i].latlng = str(this_ll)
        
        

218
[46.01349769999999, 7.8006047]
219
[46.0115108, 7.7913394]
220
[46.0056, 7.7857]
221
[46.39501, 7.10202]
222
[46.0150833, 7.772596599999999]
223
[46.159722, 7.283888999999999]
224
[46.818188, 8.227511999999999]
225
[46.4445755, 7.8813103]
226
[46.818188, 8.227511999999999]
228
[46.4572209, 8.168681099999999]
229
[46.818188, 8.227511999999999]
230
[46.818188, 8.227511999999999]
231
[46.818188, 8.227511999999999]
232
[46.107933, 7.2719435]
233
[46.4415857, 8.106052499999999]
234
None
235
None
236
[46.4887031, 8.0530777]
237
[46.4408333, 8.0919444]
227
[46.050484, 7.480085399999999]


## Keep target lakes in geojson

In [131]:
saved_geo = []
for i in range(len(gj['features'])):
    if(gj['features'][i]['properties']['name'] in list(data['Lake'])):
        saved_geo.append(gj['features'][i])

In [132]:
len(saved_geo)

202

In [133]:
gj['features'] = saved_geo

In [134]:
with open('target_lakes.geojson', 'w') as f:   
    geojson.dump(gj, f)

## Lakes per canton

In [136]:
con.groupby('Canton').size()

Canton
Appenzell Innerrhoden     3
Bern                     22
Fribourg                  2
Glarus                    7
Graubünden               63
Graubünden/Ticino         1
Jura                      1
Neuchâtel                 1
Nidwalden                 2
Obwalden                  5
Schwyz                    4
St. Gallen                8
Ticino                   33
Ticino/Valais             1
Uri                       9
Valais                   68
Vaud                      7
dtype: int64

# Seperate data for each canton for web rendering

In [2]:
all_lakes = pd.read_csv("full_data.csv")

In [3]:
all_lakes.head()

,lake_id,Lake,Canton,Elevation,Area,is_lake_link,link,latlng,with_geojson,description,...,water_volume,Region,Date,Altitude,pH,Conductivity,Temperature,O2,16S,18S
0,1,Theodulgletschersee,Valais,2851,7.76,False,https://en.wikipedia.org///wiki/Theodulgletsch...,"[45.971600, 7.715315]",True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Lai d'Immez,Graubünden,2832,4.53,False,NaN,"[46.7277984, 10.1338334]",True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Lej da Prüna,Graubünden,2815,4.03,False,NaN,"[46.477777, 9.973627]",True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Lej da Pischa,Graubünden,2770,5.05,False,NaN,"[46.47142909999999, 9.9790387]",True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Chaltwassersee,Valais,2756,4.41,False,NaN,"[46.25839999999999, 8.078289999999999]",True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
cantons_name = ["Appenzell Innerrhoden",
"Bern",
"Fribourg",
"Glarus",
"Graubünden",
"Ticino",
"Jura",
"Neuchâtel",
"Nidwalden",
"Obwalden",
"Schwyz",
"St. Gallen",
"Uri",
"Valais",
"Vaud",
"Luzern",
"Zug",
"Solothurn",
"Basel-Stadt",
"Basel-Landschaft",
"Schaffhausen",
"Appenzell Ausserrhoden",
"Aargau",
"Thurgau",
"Genève",
"Zürich"]

cantons_id = ["AI",
"BE",
"FR",
"GL",
"GR",
"TI",
"JU",
"NE",
"NW",
"OW",
"SZ",
"SG",
"UR",
"VS",
"VD",
"LU",
"ZG",
"SO",
"BS",
"BL",
"SH",
"AR",
"AG",
"TG",
"GE",
"ZH"]

In [5]:
dfc = {}
for i in range(len(cantons_id)):
    dfc[cantons_id[i]] = all_lakes[all_lakes["Canton"].str.contains(cantons_name[i])]
    dfc[cantons_id[i]].to_csv(cantons_id[i]+'.csv',index=False)

# Seperate lake geojson

In [12]:
import geojson
with open('target_lakes.geojson') as f:
    gj = geojson.load(f)

In [34]:
saved_geo = {}
for i in range(len(gj['features'])):
    if(gj['features'][i]['properties']['name'] in list(all_lakes['Lake'])):
        k = all_lakes[all_lakes["Lake"]==gj['features'][i]['properties']['name']].iloc[0]["lake_id"]
        if(k in saved_geo):
            saved_geo[k].append(gj['features'][i])
        else:
            saved_geo[k] = [gj['features'][i]]

In [38]:
for key in saved_geo:
    gj['features'] = saved_geo[key]
    with open(str(key)+'.geojson', 'w') as f:   
        geojson.dump(gj, f)
    